### CMoney_正負向標示(含斷詞)
Version：20220713  
Notes：  
 > 1. 新增正負向標示(根據正負向詞字典)
 > 2. 修正詞頻重複寫入bug
 > 3. 尚未使用到WordCloud，暫時不放入程式中
* * *
**下方首行可做參數修改**  

In [ ]:
Article_FileName = "社群文章0322" #文章檔案名稱
CkipResults_FileName = "社群文章0322" #儲存斷詞檔案名稱
CkipDict = "ckip_dict" #Ckiptagger字典檔案名稱
PositiveDict = "positive_dict" #正向詞字典檔案名稱
NegativeDict = "negative_dict" #負向詞字典檔案名稱

In [ ]:
from ckiptagger import WS, POS, NER
import ckiptagger.model_pos
import json
import jieba
import re
import csv
from csv import reader
from tqdm import tqdm
import pandas as pd

dict_path = "./dict/" #字典路徑
model_path = "./model/" #Ckiptagger model路徑

#================================Ckiptagger================================

def Ckip_Cut(articleList):
    #詞性列表 http://ckipsvr.iis.sinica.edu.tw/papers/category_list.pdf 
    #pos_list = ['Da', 'Dfa', 'Dfb', 'Di', 'Na', 'Nb', 'Nc', 'Ncd', 'Nd', 'Nv', 'A', 'VA', 'VAC', 'VB', 'VC', 'VD', 'VE', 'VF', 'VH', 'VHC', 'VI', 'VK'] #原程式詞性篩選
    #pos_list = ['Na', 'Nb', 'VA', 'VC', 'VLC', 'VD', 'VH', 'VHC'] #16分詞性
    #pos_list = ['A', 'Caa', 'Cab', 'Cba', 'Cbb', 'D', 'Da', 'Dfa', 'Dfb', 'Di', 'Dk' , 'FW', 'Na', 'Nb', 'Nc', 'Ncd', 'Nd', 'Nep', 'Neqa', 'Neqb', 'Nes', 'Neu', 'Nf', 'Ng', 'Nh', 'Nv', 'P', 'VA', 'VAC', 'VB', 'VC', 'VCL', 'VD', 'VE', 'VF', 'VG', 'VH', 'VHC', 'VI', 'VJ', 'VK', 'VL'] #基礎詞性篩選(標點符號、有、是、的、語助詞T、感嘆詞I)  
    pos_list = ['A', 'Da', 'Dfa', 'Dfb', 'FW', 'Na', 'Nb', 'Nc', 'Ncd', 'Nd', 'Nv', 'VA', 'VAC', 'VB', 'VC', 'VCL', 'VD', 'VE', 'VF', 'VH', 'VHC', 'VI', 'VJ', 'VK', 'VL'] #20220603篩選
    
    positive_csv = pd.read_csv(dict_path + PositiveDict + ".csv") #讀取正向詞字典
    negative_csv = pd.read_csv(dict_path + NegativeDict + ".csv") #讀取負向詞字典
    positive_list = positive_csv["正向詞"].to_list()
    negative_list = negative_csv["負向詞"].to_list()
    
    with open(dict_path + CkipDict +".json", "r", encoding = 'utf-8') as d: #讀取字典，字典可自定義
        ckip_dict = json.load(d)
        
    dictionary = ckiptagger.construct_dictionary(ckip_dict) #設定字典格式為指定格式
    
    #Load model with GPU
    ws = WS(model_path, disable_cuda = False) #WS(斷詞)
    pos = POS(model_path, disable_cuda = False) #POS(詞性標註)
    ner = NER(model_path, disable_cuda = False) #NER(實體辨識)
    
    #處理斷詞
    ws_results = ws(articleList, coerce_dictionary = dictionary)
    
    #處理詞性標註
    pos_results = pos(ws_results)
    
    with open("./" + CkipResults_FileName + "_ckip_results.csv",'w' , newline = '', encoding = 'utf_8_sig') as f: 
        csv_write = csv.writer(f)
        csv_write.writerow(["詞性", "斷詞"]) #斷詞結果先寫入第一列標題 
    ckip_results = [] #儲存斷詞結果
    for i in range(len(ws_results)):
        remainderWords = [] #僅儲存要的斷詞結果
        for j in range(len(ws_results[i])):
            if pos_results[i][j] in pos_list: #僅擷取有在詞性列表中的詞
                with open("./" + CkipResults_FileName + "_ckip_results.csv",'a' , newline = '', encoding = 'utf_8_sig') as f:
                    csv_write = csv.writer(f)
                    csv_write.writerow([pos_results[i][j], ws_results[i][j]]) #寫入斷詞及詞性資料
                remainderWords.append(ws_results[i][j])
        ckip_results.append(remainderWords)
    
    #處理正負向標示
    with open("./" + CkipResults_FileName + "_positive_results.csv",'w' , newline = '', encoding = 'utf_8_sig') as f: 
        csv_write = csv.writer(f)
        csv_write.writerow(["正向詞"]) #先寫入第一列標題
    positive_results = [] #儲存正向詞結果
    for i in range(len(ws_results)):
        positiveWords = [] #僅儲存要的正向詞結果
        for j in range(len(ws_results[i])):
            if ws_results[i][j] in positive_list: #僅擷取有在正向詞字典中的詞
                with open("./" + CkipResults_FileName + "_positive_results.csv",'a' , newline = '', encoding = 'utf_8_sig') as f:
                    csv_write = csv.writer(f)
                    csv_write.writerow([ws_results[i][j]]) #寫入正向詞
                positiveWords.append(ws_results[i][j])
        positive_results.append(positiveWords)
    
    with open("./" + CkipResults_FileName + "_negative_results.csv",'w' , newline = '', encoding = 'utf_8_sig') as f: 
        csv_write = csv.writer(f)
        csv_write.writerow(["負向詞"]) #先寫入第一列標題
    negative_results = [] #儲存負向詞結果
    for i in range(len(ws_results)):
        negativeWords = [] #僅儲存要的負向詞結果
        for j in range(len(ws_results[i])):
            if ws_results[i][j] in negative_list: #僅擷取有在負向詞字典中的詞
                with open("./" + CkipResults_FileName + "_negative_results.csv",'a' , newline = '', encoding = 'utf_8_sig') as f:
                    csv_write = csv.writer(f)
                    csv_write.writerow([ws_results[i][j]]) #寫入負向詞
                negativeWords.append(ws_results[i][j])
        negative_results.append(negativeWords)
    
    #處理實體辨識
    ner_results = ner(ws_results, pos_results)
    
    with open("./" + CkipResults_FileName + "_ner_results.csv",'w' , newline = '', encoding = 'utf_8_sig') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(["類別", "實體辨識"]) #寫入實體辨識第一列標題
    df_articles_data = pd.read_csv("./" + Article_FileName + ".csv")
    df_articles_data.dropna(subset = ['文章內容'], inplace = True)
    for n in range(len(df_articles_data)):
        list_ner_results = list(ner_results.pop()) #抓取最後一筆文章的實體辨識
        for i in range(len(list_ner_results)):
            with open("./" + CkipResults_FileName + "_ner_results.csv",'a' , newline = '', encoding = 'utf_8_sig') as f:
                csv_write = csv.writer(f)
                csv_write.writerow([list_ner_results[i][2], list_ner_results[i][3]]) #寫入類別及實體辨識資料
    
    del ws 
    del pos
    del ner #釋放內存
    
    return ckip_results

#================================Jieba================================

def Jieba_Cut(articleList):
    stopWords = [] #儲存停用詞
    remainderWords = [] #儲存剩餘要的詞

    with open('dict/stopWords.txt', 'r', encoding = 'UTF-8-sig') as f: #取停用詞
        for data in f.readlines():
            data = data.strip()
            stopWords.append(data)
    
    jieba.load_userdict('dict/jb_dict.txt') #取字典

    jieba_results = [] ##儲存斷詞結果
    for article in articleList:
        try:
            atc = re.sub(r'\W|\s|\d|[a-zA-z]', '', article) #刪除數字、英文等
            atc = re.sub(r'[^\u4e00-\u9fa5]', '', article) #只留中文
        except:
            input('Error\n' + str(article))
        
        words = jieba.cut(atc, cut_all = False) #處理斷詞
        remainderWords = list(filter(lambda a: a not in stopWords and a != '\n' and a != ' ', words)) #篩選掉停用詞、空白、空行
        jieba_results.append(remainderWords)
        
    return jieba_results

### 讀資料並刪除空白文章內容列(rows)

In [ ]:
df_articles_data = pd.read_csv("./" + Article_FileName + ".csv")
df_articles_data.dropna(subset = ['文章內容'], inplace = True)
df_articles_data.head(10)


### 資料整理為list

In [ ]:
articles_data = df_articles_data['文章內容'].to_list()
articles_data[:1]

### Jieba(測試用)
因Jieba斷詞速度較快，做為資料斷詞測試用

In [ ]:
import time
start = time.time() #計算程式執行時間用
jieba_results = Jieba_Cut(articles_data)
end = time.time() #計算程式執行時間用

print("斷詞執行時間共：", end - start, "秒") #計算程式執行時間用
print("不精準的預估Ckiptagger所需執行時間最多約為：", (end - start) * 1100, "秒")

jieba_results[:1]

### CkipTagger
因效率較差，處理速度很慢是正常的...  
可參考上行預估Ckiptagger所需執行時間

In [ ]:
import time
start = time.time() #計算程式執行時間用
ckip_results = Ckip_Cut(articles_data)
end = time.time() #計算程式執行時間用

df_ckip_results = pd.read_csv("./" + CkipResults_FileName + "_ckip_results.csv")

print("斷詞的資料筆數共有", len(df_ckip_results), "筆")
print("斷詞執行時間共：", end - start, "秒") #計算程式執行時間用
print("斷詞檔案名稱：" + CkipResults_FileName + "_ckip_results.csv")
print("實體辨識檔案名稱：" + CkipResults_FileName + "_ner_results.csv")
print("正向詞標示檔案名稱：" + CkipResults_FileName + "_positive_results.csv")
print("負向詞標示檔案名稱：" + CkipResults_FileName + "_negative_results.csv")

### 詞頻(斷詞、正負向詞)

In [ ]:
from collections import Counter
temp = []
for i in ckip_results:
    temp += i
wordcount = Counter(temp)

with open("./" + CkipResults_FileName + "_wordcount.csv",'w' , newline = '', encoding = 'utf_8_sig') as f: 
        csv_write = csv.writer(f)
        csv_write.writerow(["斷詞詞頻", "次數"]) #先寫入第一列標題
for i in range (len(wordcount.most_common())):
    with open("./" + CkipResults_FileName + "_wordcount.csv",'a' , newline = '', encoding = 'utf_8_sig') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(wordcount.most_common()[i])

wordcount.most_common(10)

In [ ]:
from collections import Counter
temp = []

positive_csv = pd.read_csv("./" + CkipResults_FileName + "_positive_results.csv") #讀取正向詞結果
positive_list = positive_csv["正向詞"].to_list()

for i in positive_list:
    temp += [i]
wordcount = Counter(temp)

with open("./" + CkipResults_FileName + "_positive_wordcount.csv",'w' , newline = '', encoding = 'utf_8_sig') as f: 
        csv_write = csv.writer(f)
        csv_write.writerow(["正向詞詞頻", "次數"]) #先寫入第一列標題
for i in range (len(wordcount.most_common())):
    with open("./" + CkipResults_FileName + "_positive_wordcount.csv",'a' , newline = '', encoding = 'utf_8_sig') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(wordcount.most_common()[i])

wordcount.most_common(10)

In [ ]:
from collections import Counter
temp = []

negative_csv = pd.read_csv("./" + CkipResults_FileName + "_negative_results.csv") #讀取負向詞結果
negative_list = negative_csv["負向詞"].to_list()

for i in negative_list:
    temp += [i]
wordcount = Counter(temp)

with open("./" + CkipResults_FileName + "_negative_wordcount.csv",'w' , newline = '', encoding = 'utf_8_sig') as f: 
        csv_write = csv.writer(f)
        csv_write.writerow(["負向詞詞頻", "次數"]) #先寫入第一列標題
for i in range (len(wordcount.most_common())):
    with open("./" + CkipResults_FileName + "_negative_wordcount.csv",'a' , newline = '', encoding = 'utf_8_sig') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(wordcount.most_common()[i])

wordcount.most_common(10)